# Instructions

This script is what we used to create our Logistic Regression models for the top 5 most common topics. It can be run out-of-the-box, and the models will be saved in the current directory, namely `Model Generation Scripts`

Note that the code is somewhat agnostic to model type, and it is trivial to substitute a different sklearn model, e.g. RandomForest, KNN, etc.

# Code

## Imports and Constants

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
import pickle

### The Hebrew stopwords set we used

In [16]:
HEBREW_STOPWORDS_1 = ['אני',
'את',
'אתה',
'אנחנו',
'אתן',
'אתם',
'הם',
'הן',
'היא',
'הוא',
'שלי',
'שלו',
'שלך',
'שלה',
'שלנו',
'שלכם',
'שלכן',
'שלהם',
'שלהן',
'לי',
'לו',
'לה',
'לנו',
'לכם',
'לכן',
'להם',
'להן',
'אותה',
'אותו',
'זה',
'זאת',
'אלה',
'אלו',
'תחת',
'מתחת',
'מעל',
'בין',
'עם',
'עד',
'נגר',
'על',
'אל',
'מול',
'של',
'אצל',
'כמו',
'אחר',
'אותו',
'בלי',
'לפני',
'אחרי',
'מאחורי',
'עלי',
'עליו',
'עליה',
'עליך',
'עלינו',
'עליכם',
'לעיכן',
'עליהם',
'עליהן',
'כל',
'כולם',
'כולן',
'כך',
'ככה',
'כזה',
'זה',
'זות',
'אותי',
'אותה',
'אותם',
'אותך',
'אותו',
'אותן',
'אותנו',
'ואת',
'את',
'אתכם',
'אתכן',
'איתי',
'איתו',
'איתך',
'איתה',
'איתם',
'איתן',
'איתנו',
'איתכם',
'איתכן',
'יהיה',
'תהיה',
'היתי',
'היתה',
'היה',
'להיות',
'עצמי',
'עצמו',
'עצמה',
'עצמם',
'עצמן',
'עצמנו',
'עצמהם',
'עצמהן',
'מי',
'מה',
'איפה',
'היכן',
'במקום שבו',
'אם',
'לאן',
'למקום שבו',
'מקום בו',
'איזה',
'מהיכן',
'איך',
'כיצד',
'באיזו מידה',
'מתי',
'בשעה ש',
'כאשר',
'כש',
'למרות',
'לפני',
'אחרי',
'מאיזו סיבה',
'הסיבה שבגללה',
'למה',
'מדוע',
'לאיזו תכלית',
'כי',
'יש',
'אין',
'אך',
'מנין',
'מאין',
'מאיפה',
'יכל',
'יכלה',
'יכלו',
'יכול',
'יכולה',
'יכולים',
'יכולות',
'יוכלו',
'יוכל',
'מסוגל',
'לא',
'רק',
'אולי',
'אין',
'לאו',
'אי',
'כלל',
'נגד',
'אם',
'עם',
'אל',
'אלה',
'אלו',
'אף',
'על',
'מעל',
'מתחת',
'מצד',
'בשביל',
'לבין',
'באמצע',
'בתוך',
'דרך',
'מבעד',
'באמצעות',
'למעלה',
'למטה',
'מחוץ',
'מן',
'לעבר',
'מכאן',
'כאן',
'הנה',
'הרי',
'פה',
'שם',
'אך',
'ברם',
'שוב',
'אבל',
'מבלי',
'בלי',
'מלבד',
'רק',
'בגלל',
'מכיוון',
'עד',
'אשר',
'ואילו',
'למרות',
'אס',
'כמו',
'כפי',
'אז',
'אחרי',
'כן',
'לכן',
'לפיכך',
'מאד',
'עז',
'מעט',
'מעטים',
'במידה',
'שוב',
'יותר',
'מדי',
'גם',
'כן',
'נו',
'אחר',
'אחרת',
'אחרים',
'אחרות',
'אשר',
'או'
]



## Load Cleaned Training Data

In [26]:
CLEANED_DATA_PATH = '../../Data/Cleaned Data/good_df.json'
good_df = pd.read_json(CLEANED_DATA_PATH)
good_df = good_df[['text','pm_ref', 'topic']]
good_df.head()

,text,pm_ref,topic
0,וכלי חרש אשר יגע בו הזב ישבר וכל כלי עץ ישטף ב...,Leviticus 15:13,טבילה
1,והיה לפנות ערב ירחץ במים וכבא השמש יבא אל תוך ...,Deuteronomy 23:12,טבילה
2,אמר לו רבי יהודה והלא כהנים מבעוד יום הם טובלים,Berakhot 2b:13,טבילה
3,והתנן טמא שירד לטבול ספק טבל ספק לא טבל ואפילו...,Eruvin 35b:2,טבילה
4,דאמר רב יהודה אמר שמואל כל המצוות מברך עליהן ע...,Pesachim 7b:9-12,טבילה


## Helper Functions

### Build Topic-Sensitive Positive/Negative Datasets


In [18]:
def create_single_topic_df(fixed_data, topic, random_state=613, loc ='fixed_a_text'):
  using = fixed_data.copy()
  using['is_target'] = np.where(using['topic']==(topic), 1,0)
  positive = using[using['is_target']==1]
  using = using[using[loc].isin(positive[loc]) == False]
  negative = using.sample(len(positive.index), random_state=random_state)
  combined = pd.concat([positive, negative], axis=0)
  print("%s, number of topics %d" % (topic, positive['is_target'].sum() ))
  return combined

### Preprocessing (Vectorize Data and Train/Test Split)

In [19]:
def split_and_transform(single_topic_df, vct=CountVectorizer, bag_size=10_000, stopwords=None, random_state=613, loc ='fixed_a_text'):
  texts = single_topic_df[loc]
  targets = single_topic_df['is_target'].to_numpy()
  train_text, test_text, train_label, test_label = train_test_split(texts.to_numpy(), targets, random_state=random_state, test_size=0.2, stratify=targets)
  vectorizer = vct(max_features=bag_size, stop_words=stopwords)
  
  # only fit vectorizer on training data
  train_bag = vectorizer.fit_transform(train_text)
  test_bag = vectorizer.transform(test_text)
  pickle.dump(vectorizer, open(t+"_bag.pkl","wb"))
  return train_bag, test_bag, train_label, test_label

### Train an `sklearn` model

In [20]:
def train_sklearny(clf, train_bag, train_label):
  clf.fit(train_bag, train_label)
  return clf

### Evaluate an `sklearn` model

In [21]:
def evaluate_sklearny(clf, test_bag, test_label, verbose=True):
  res = ''
  predictions = clf.predict(test_bag)
  res = res + f'\nscore: {clf.score(test_bag,test_label)}'
  if verbose: res = res + f'\nClassification Report: \n{classification_report(test_label, predictions)}'
  return res

## Model Training and Export

Picking the top 5 most common topics:

In [22]:
topics = good_df.value_counts('topic').keys().tolist()[:5]
topics

['למוד', 'תורה', 'תפלה', 'תשובה', 'ישראל']

The pre-processing/train/evaluate/save loop for each topic:

In [23]:
for t in topics: 
  single_topic_df = create_single_topic_df(good_df, t, 613, 'text')
  train_bag, test_bag, train_label, test_label = split_and_transform(
      single_topic_df, vct=CountVectorizer, bag_size=10_000, stopwords=HEBREW_STOPWORDS_1, loc= 'text')
  model = train_sklearny(LogisticRegression(), train_bag, train_label)
  eval = evaluate_sklearny(model, test_bag, test_label, verbose=True)
  print(eval)
  pickle.dump(model, open(t+".pkl","wb"))

למוד, number of topics 998


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['באיזו', 'בו', 'במקום', 'בשעה', 'הסיבה', 'לאיזו', 'למקום', 'מאיזו', 'מידה', 'מקום', 'סיבה', 'שבגללה', 'שבו', 'תכלית'] not in stop_words.
  warnings.warn(



score: 0.8475
Classification Report: 
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       200
           1       0.86      0.83      0.85       200

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

תורה, number of topics 894


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['באיזו', 'בו', 'במקום', 'בשעה', 'הסיבה', 'לאיזו', 'למקום', 'מאיזו', 'מידה', 'מקום', 'סיבה', 'שבגללה', 'שבו', 'תכלית'] not in stop_words.
  warnings.warn(



score: 0.8854748603351955
Classification Report: 
              precision    recall  f1-score   support

           0       0.83      0.97      0.89       179
           1       0.97      0.80      0.87       179

    accuracy                           0.89       358
   macro avg       0.90      0.89      0.88       358
weighted avg       0.90      0.89      0.88       358

תפלה, number of topics 837


...


score: 0.8507462686567164
Classification Report: 
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       168
           1       0.89      0.80      0.84       167

    accuracy                           0.85       335
   macro avg       0.85      0.85      0.85       335
weighted avg       0.85      0.85      0.85       335

תשובה, number of topics 711


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['באיזו', 'בו', 'במקום', 'בשעה', 'הסיבה', 'לאיזו', 'למקום', 'מאיזו', 'מידה', 'מקום', 'סיבה', 'שבגללה', 'שבו', 'תכלית'] not in stop_words.
  warnings.warn(



score: 0.8771929824561403
Classification Report: 
              precision    recall  f1-score   support

           0       0.83      0.96      0.89       143
           1       0.95      0.80      0.87       142

    accuracy                           0.88       285
   macro avg       0.89      0.88      0.88       285
weighted avg       0.89      0.88      0.88       285

ישראל, number of topics 695


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['באיזו', 'בו', 'במקום', 'בשעה', 'הסיבה', 'לאיזו', 'למקום', 'מאיזו', 'מידה', 'מקום', 'סיבה', 'שבגללה', 'שבו', 'תכלית'] not in stop_words.
  warnings.warn(



score: 0.8165467625899281
Classification Report: 
              precision    recall  f1-score   support

           0       0.79      0.86      0.82       139
           1       0.85      0.77      0.81       139

    accuracy                           0.82       278
   macro avg       0.82      0.82      0.82       278
weighted avg       0.82      0.82      0.82       278

